In [1]:
import spacy
import pandas as pd
!python3 -m spacy download pt_core_news_lg
pln = spacy.load('pt_core_news_lg')
pln

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
caminho_arquivo = '/content/drive/MyDrive/TCC - UFMS/CSBC-WICS/Corpus/corpus.csv'

dataframe = pd.read_csv(caminho_arquivo)

# Filtrar o DataFrame para excluir as letras do gênero gospel
dataframe_sem_gospel = dataframe[dataframe['genre'] != 'gospel']

# Exibir as primeiras linhas do DataFrame filtrado
dataframe_sem_gospel.head()


,artist_name,genre,music_title,music_lyric
0,Black Style,axe,Amassa a Latinha Com a Bunda,Eu tô trazendo a novidade em salvador\n(eu tô ...
1,Black Style,axe,Vaco Vuco,Você marcou comigo pra sair\nMas já pensou se ...
2,Black Style,axe,"Tome, Todinha",Que mulher gulosa\nEla entra na roda e não que...
3,Black Style,axe,Toma Que É de Graça,"Toma, toma meu carinho\ntoma meu amor e a minh..."
4,Black Style,axe,Toda de Mel,"Foi esculpida,escolhida,desenhada no papel\nNã..."


In [4]:
dataframe_sem_gospel.shape[0]

105024

Árvore de Dependência - Masculino

In [5]:
from spacy.matcher import DependencyMatcher

dep_matcher = DependencyMatcher(vocab=pln.vocab)

vozpassiva = [{"RIGHT_ID": "verbo", "RIGHT_ATTRS": {"MORPH": {"IN": ["Gender=Masc|Number=Sing|VerbForm=Part|Voice=Pass", "Gender=Masc|Number=Plur|VerbForm=Part|Voice=Pass"]}}},
              {"LEFT_ID": "verbo", "REL_OP": ">", "RIGHT_ID": "sujeito", "RIGHT_ATTRS": {"DEP": "nsubj:pass", "LOWER": {"IN": ["homem", "homens", "ele", "eles", "menino", "meninos", "garoto", "garotos", "senhor", "senhores", "moços", "moços", "rapaz", "rapazes", "joão", "cavalheiro", "cavalheiros", "rei", "reis", "esposo", "esposos", "marido", "maridos", "namorado", "namorados", "você", "vocês"]}}},
              {"LEFT_ID": "verbo", "REL_OP": ">", "RIGHT_ID": "auxliar", "RIGHT_ATTRS": {"DEP": "aux:pass", "POS":"AUX"}}]

sujeitoauxadjetivo = [{"RIGHT_ID": "adjetivo", "RIGHT_ATTRS": {"POS": "ADJ"}},
                      {"LEFT_ID": "adjetivo", "REL_OP": ">", "RIGHT_ID": "sujeito", "RIGHT_ATTRS": {"DEP": "nsubj", "LOWER": {"IN": ["homem", "homens", "ele", "eles", "menino", "meninos", "garoto", "garotos", "senhor", "senhores", "moços", "moços", "rapaz", "rapazes", "joão", "cavalheiro", "cavalheiros", "rei", "reis", "esposo", "esposos", "marido", "maridos", "namorado", "namorados"]}}},
                      {"LEFT_ID": "adjetivo", "REL_OP": ">", "RIGHT_ID": "auxiliar", "RIGHT_ATTRS": {"POS":"AUX"}}]

voceauxadjetivo = [{"RIGHT_ID": "adjetivo", "RIGHT_ATTRS": {"POS": "ADJ", "MORPH":{"IN":["Gender=Masc|Number=Sing", "Gender=Masc|Number=Plur"]}}},
                   {"LEFT_ID": "adjetivo", "REL_OP": ">", "RIGHT_ID": "sujeito", "RIGHT_ATTRS": {"DEP": "nsubj", "LOWER": {"in": ["você", "vocês"]}}},
                   {"LEFT_ID": "adjetivo", "REL_OP": ">", "RIGHT_ID": "auxiliar", "RIGHT_ATTRS": {"POS":"AUX"}}]

ficar_parecer_tornar = [{"RIGHT_ID": "verbo", "RIGHT_ATTRS": {"LEMMA":
                          {"IN": ["parecer", "ficar", "tornar"]}}},
                        {"LEFT_ID": "verbo", "REL_OP": ">", "RIGHT_ID": "sujeito", "RIGHT_ATTRS": {"DEP": "nsubj", "LOWER": {"IN": ["homem", "homens", "ele", "eles", "menino", "meninos", "garoto", "garotos", "senhor", "senhores", "moços", "moços", "rapaz", "rapazes", "joão", "cavalheiro", "cavalheiros", "rei", "reis", "esposo", "esposos", "marido", "maridos", "namorado", "namorados"]}}},
                        {"LEFT_ID": "verbo", "REL_OP":">", "RIGHT_ID": "adjetivo", "RIGHT_ATTRS": {"DEP":"xcomp", "POS": "ADJ"}}]

voce_ficar_parecer_tornar = [{"RIGHT_ID": "verbo", "RIGHT_ATTRS": {"LEMMA":
                                {"IN": ["parecer", "ficar", "tornar"]}}},
                             {"LEFT_ID": "verbo", "REL_OP": ">", "RIGHT_ID": "sujeito", "RIGHT_ATTRS": {"DEP": "nsubj", "LOWER": {"IN": ["você", "vocês"]}}},
                             {"LEFT_ID": "verbo", "REL_OP":">", "RIGHT_ID": "adjetivo", "RIGHT_ATTRS": {"DEP":"xcomp", "POS": "ADJ", "MORPH":{"IN":["Gender=Masc|Number=Sing", "Gender=Masc|Number=Plur"]}}}]

sujeitoadjetivo = [{"RIGHT_ID": "noun", "RIGHT_ATTRS": {"LOWER": {"IN": ["homem", "homens", "ele", "eles", "menino", "meninos", "garoto", "garotos", "senhor", "senhores", "moços", "moços", "rapaz", "rapazes", "joão", "cavalheiro", "cavalheiros", "rei", "reis", "esposo", "esposos", "marido", "maridos", "namorado", "namorados"]}}},
                   {"LEFT_ID": "noun", "REL_OP":">", "RIGHT_ID": "adjetivo", "RIGHT_ATTRS": {"DEP":"amod", "POS": "ADJ"}}]

dep_matcher.add('vozpassiva', patterns=[vozpassiva])
dep_matcher.add('ficar_parecer_tornar', patterns=[ficar_parecer_tornar])
dep_matcher.add('sujeitoauxadjetivo', patterns=[sujeitoauxadjetivo])
dep_matcher.add('sujeitoadjetivo', patterns=[sujeitoadjetivo])
dep_matcher.add('voceauxadjetivo', patterns=[voceauxadjetivo])
dep_matcher.add('voce_ficar_parecer_tornar', patterns=[voce_ficar_parecer_tornar])


In [6]:
resultados = []

for index, row in dataframe_sem_gospel.iterrows():
    text = str(row['music_lyric'])
    doc = pln(text)
    dep_matches = dep_matcher(doc)
    dep_matcher
    for match in dep_matches:
      pattern_name = match[0]
      matches = match[1]
      if pln.vocab[pattern_name].text == 'vozpassiva' or pln.vocab[pattern_name].text == 'voceauxadjetivo' or pln.vocab[pattern_name].text == 'sujeitoauxadjetivo':
        inicio, final = matches[1], matches[0]
        resultados.append({'Padrão': pln.vocab[pattern_name].text, 'Frase': doc[inicio:final+1], 'Letra_Inteira': text})
      elif pln.vocab[pattern_name].text == 'ficar_parecer_tornar' or pln.vocab[pattern_name].text == 'voce_ficar_parecer_tornar':
        inicio, final = matches[1], matches[2]
        resultados.append({'Padrão': pln.vocab[pattern_name].text, 'Frase': doc[inicio:final+1], 'Letra_Inteira': text})
      elif pln.vocab[pattern_name].text == 'sujeitoadjetivo':
        inicio, final = matches[0], matches[1]
        resultados.append({'Padrão': pln.vocab[pattern_name].text, 'Frase': doc[inicio:final+1], 'Letra_Inteira': text})

resultados_df = pd.DataFrame(resultados)
resultados_df.head()

,Padrão,Frase,Letra_Inteira
0,sujeitoauxadjetivo,(),"Quer colar comigo, comigo não cola, pra colar ..."
1,voceauxadjetivo,"(você, tá, louco)","Quer colar comigo, comigo não cola, pra colar ..."
2,voceauxadjetivo,"(você, tá, louco)","Quer colar comigo, comigo não cola, pra colar ..."
3,vozpassiva,"(Vocês, foram, parados)",Walls of Jericho - Muros de Jericó\nVocê sabe ...
4,voceauxadjetivo,"(Vocês, estão, todos, fracos)",Walls of Jericho - Muros de Jericó\nVocê sabe ...


In [7]:
from google.colab import files

resultados_df = pd.DataFrame(resultados)
resultados_df.to_csv('resultados_arvore_dependencia_masculino.csv', index=False)
files.download('resultados_arvore_dependencia_masculino.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>